## GPT-2 Medium via PyTorch

This is the original iteration of the text generation model.

This initial version uses PyTorch and the HuggingFace transformers library to manually retrain the GPT-2 Medium model. 

The model experienced difficulty with overfitting and text quality. After some experimentation, this version was scrapped in favor of the smaller model created through GPT-2 Simple and TensorFlow which proved more efficient to tune and adjust. 

The notebook was created using Google Colab and borrows heavily from Charles Pierse's [Script Buddy](https://github.com/cdpierse/script_buddy_v2) and Martin Frolov's [Joke Generator](https://gist.github.com/mf1024/3df214d2f17f3dcc56450ddf0d5a4cd7)

## Install and Import

In [ ]:
! pip install transformers 
! pip install torch

In [ ]:
# GPT and PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup


# Logging and Warnings
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings('ignore')

# Data manipulation, Misc.
import random
import pandas as pd
import numpy as np

## Load Data

We load our dataset from Google Drive and create a PyTorch DataLoader class which will handle generating and passing our training batchs. 

In [ ]:
# load in complete dataset, contains several columns
anime_df = pd.read_csv('drive/My Drive/Anime.csv')
anime_df.rename(str.strip, axis='columns', inplace=True)
# reduce to target column of synopsis text
synopsis_list = anime_df['synopsis']

In [ ]:
# DataLoader pbject which will add start and end of text tokens and batch load our data
class SynposisDataset(Dataset):
    def __init__(self):
        super().__init__()
        
        data = synopsis_list

        self.syn_list = []
        self.end_of_text_token = "<|endoftext|>"

        for idx, synopsis in enumerate(data):
            syn_str = f"{self.end_of_text_token}START:{synopsis}{self.end_of_text_token}"
            # syn_str = f"{self.end_of_text_token} {synopsis}{self.end_of_text_token}"
            self.syn_list.append(syn_str)
            
            
    def __len__(self):
        return len(self.syn_list)

    def __getitem__(self, item):
        return self.syn_list[item]


In [ ]:
# intialize loader and specify batch size
dataset = SynposisDataset()
synopsis_loader = DataLoader(dataset, batch_size=1, shuffle=True)

## Prepare GPT-2

Here we'll set the parameters and processing settings then load the GPT-2 tokenizer and model.

In [ ]:
"""
Model Settings
"""
BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 2e-5
WARMUP_STEPS = 10000
# MAX_SEQ_LEN = 550


# Use parallel GPU computation if available
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [ ]:
# GPT-specific text tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
# load GPT-2 medium model
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

## Model Training

Here we repeatedly feed the model batches of training data with itself as the target label -- GPT-2 is training to predict each training sequence one token at a time. 

We track the loss and output a generated sample every 200 training steps from a random int which serves as the seed token. 

We experimented with a few implementations of this, but were'nt able to find a satisfactory set of parameters to prevent overfitting while retaining the structure of the target data. The model quickly goes from heavily underfit (producing verbatim pre-training text) to heavily overfit (producing verbatim training data) or produces something closer to the latter but lacking in coherence.


In [ ]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE) # gradient descent optimizer
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1) # learning rate scheduler
synopsis_count = 0 
sum_loss = 0.0
batch_count = 0

for epoch in range(EPOCHS):
    for _ , synopsis in enumerate(synopsis_loader):
        # feed data
        syn_tens = torch.tensor(tokenizer.encode(synopsis[0])).unsqueeze(0).to(device)
        outputs = model(syn_tens, labels=syn_tens)
        
        # backward pass
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().item()

        # adjust weights and zero gradients      
        synopsis_count = synopsis_count + 1
        if synopsis_count == BATCH_SIZE:
            synopsis_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        # output sample    
        if batch_count == 200:
            model.eval()

            print(f"sum loss {sum_loss}")
            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )

            print("Output:\n" + 100 * '-')
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            print("\n")
            
            batch_count = 0
            sum_loss = 0.0
            model.train()

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


sum loss 871.0637943744659
Output:
----------------------------------------------------------------------------------------------------
0:  lifesassets.com (http://www.guru.com/) for technical advice and technical support.

Please contact me if you have any questions or suggestions.




Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


sum loss 784.4341342449188
Output:
----------------------------------------------------------------------------------------------------
0:  BST) and the BST-controlled signals (1) and (2) to produce stable (or, equivalently, delayed) signals that can be transferred from source to the detector. We hypothesized that a single pulse at the transmitter's frequency would result in an increase in the signal's magnitude and direction of response.

The signal produced by the BST-controlled signal (1) is dependent on the transmitter's rate-response signal as well as the detector's phase, which depends on the transmitter's rate-response system and the direction (or directionality) of the signal's transfer rate from source to detector. Thus, a pulse at a particular rate would cause the signal to be sensitive to a given transmitter frequency and direction. For simplicity, we have used a typical receiver-transmitter synchronous switching system to mimic the BST-controlled signals. The BST-controlled

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


...

sum loss 587.9781355261803
Output:
----------------------------------------------------------------------------------------------------
0:  AoST-exclusive special included with the second volume of the Blu-ray/DVD.




Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


sum loss 584.6002468168736
Output:
----------------------------------------------------------------------------------------------------
0:  RocketsSTART:Short animation by Yonezawa Tadanari.




Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


sum loss 595.24416705966
Output:
----------------------------------------------------------------------------------------------------
0:  AnimalsSTART:This is the story about the creation of a human family by the children of a farmer. (Source: AniDB)




In [ ]:
# save model
torch.save(model.state_dict(), 'drive/My Drive/ML Models/syn_gen_3_epoch.pth')

## Generator Functions

Cribbed from Martin Frolov -- unused since training samples never reached a satisfactory state

In [ ]:
# Function to first select topN tokens from the probability list and then based on the N word distribution
# select random token ID
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [ ]:
# tensor of tokenized ids
cur_ids = torch.tensor(tokenizer.encode(randanime)).unsqueeze(0).long().to(device)

model.eval() # sets model to eval mode 
with torch.no_grad(): # gradients? stop autograd from tracking history on tensors
    
    for i in range(250):
        outputs = model(cur_ids, labels=cur_ids) # instantiate GPT2LMHeadModel with our sample text
        loss, logits = outputs[:2] # track the loss and logit outputs
        softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
        next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=5) #Randomly(from the given probability distribution) choose the next word from the top n words
        cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    print(output_text)